In [1]:
# Load library
import fairsearchcore as fsc
import pandas as pd
import numpy as np


In [2]:
# PLAIN DF * 200
dataframen = pd.read_csv("TWO_samlet_preds_index_ny_PLAIN.csv") 
dataframen_input = pd.read_csv("plain_seed24_gemmes.csv") ## denne er til brug da vi skulle regne RMSE på fair

#dfs = np.array_split(dataframen, 200)
#dfs[2][:50]

In [3]:
#dataframen_input.sort_values(by='Preds', ascending=False)[:50]

In [3]:
#MERGESORT DF * 200
dataframen_merge = pd.read_csv('stort_mergesort_ranking.csv')
df_merge_use = dataframen_merge.copy()

df_merge_use = df_merge_use[['index',"Unnamed: 0", "4", "3", "18"]].rename(columns= {'index':'index',"Unnamed: 0": "Idx", "4":"Gender_bins", "3":"Preds", "18":"COL_GRADE_AVG"})
df_merge_BIG_split = np.array_split(df_merge_use, 200)


In [4]:
df_merge_BIG_split[0][:500]

,index,Idx,Gender_bins,Preds,COL_GRADE_AVG
0,1784,0,1,97.666667,98.666667
1,3721,1,0,96.666667,97.333333
2,1989,2,0,95.000000,97.000000
3,3830,3,0,93.666667,96.333333
4,2332,4,0,93.333333,98.000000
...,...,...,...,...,...
495,3622,495,1,73.666667,97.333333
496,3628,496,1,73.666667,75.000000
497,3971,497,0,73.666667,94.333333
498,4073,498,1,73.666667,91.666667


## NY FA*IR funktion: Maximize utility while maximizing fairness. 




In [8]:
dataframen_for_RMSE = dataframen

dataframen_for_RMSE.loc[dataframen_for_RMSE.Gender_bins > 0, 'Gender_bins'] = 1
dataframen_for_RMSE.loc[dataframen_for_RMSE.Gender_bins < 0, 'Gender_bins'] = 0



In [9]:
# dataframen_for_RMSE=dataframen_for_RMSE.drop(columns={"Unnamed: 0"})
# dataframen_for_RMSE=dataframen_for_RMSE.rename(columns= {'Idx': "Unnamed: 0"})

In [10]:

# GUDF = pd.read_csv('GUDF.csv')
# dataframen_for_RMSE_new = pd.concat([dataframen_for_RMSE, GUDF.STRATUM], axis=1)



In [17]:
# df_try=dataframen_for_RMSE
df_try = dataframen_for_RMSE#[:10000]


In [12]:
# If WO run this
# df_try['Gender_bins'] = df_try['Gender_bins'].str.strip('[]').astype(int)
# df_try

In [13]:
import warnings

In [14]:
def modified_FAIR(df, k, n_prefix, prop=-1 ):
    
    """" 
    Our fair algorithm inspired by Zehlike's FA*IR:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired prop for each n prefix. 
    prop is desired proportion with default value of the proportion of the protected group in the df. 
    
    Output is 'result' of a fair top-k ranking with desired proportion for the whole df and in every prefix
    
    """
    
    if prop<0:
        prop = len(df[df.Gender_bins == 1])/len(df)
        print("defualt proportios: ", prop)
        
    #we subtract by 1, because we are using 0-index
    n_prefix = int(n_prefix *k)-1

    # if k/n_prefix != int(k/n_prefix):
    #     warnings.warn('Please use 0.1 as 10 percent of the top-k as prefix')
    
    #split df in two df groups: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #protected (1=Female)
    dfp = df[df.Gender_bins == 1].sort_values(by='Preds', ascending=False).reset_index()
    #non-protected
    dfpn = df[df.Gender_bins == 0].sort_values(by='Preds', ascending=False).reset_index()
    
    #dfnew with of the top-k fair sorting of org. df
    #new df
    dfnew = pd.DataFrame(columns = df.columns)
    
    #print OBS if it's not possible to achieve the given prop for each n prefix for the whole top-k
    
    #for-loop for creating ranking in batches of prefix_n through k
    
    #desired number of protected candidates in each n batch
    prop_ok = int(n_prefix*prop)
    prop_pn = int(n_prefix - prop_ok)
        
    #index in each dataframe
    dfp_Idx = 0
    dfpn_Idx = 0
        
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        p_Idx = 0
        pn_Idx = 0
        
            
        if len(dfnew) >= k: #stop if desired k is achieved
            #print("500")
            break

        for j in range(n_prefix):
            if dfp.Preds.loc[dfp_Idx] > dfpn.Preds.loc[dfpn_Idx]:
                dfnew = dfnew.append(dfp.loc[dfp_Idx])
                p_Idx += 1
                dfp_Idx += 1
            else:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx])
                pn_Idx += 1
                dfpn_Idx += 1

            #if the proportion of nonprotected group is achieved within the prefix, add rest as protected 
            if pn_Idx == prop_pn:
                dfnew = dfnew.append(dfp.loc[dfp_Idx : dfp_Idx + (prop_ok-p_Idx)])
                dfp_Idx += (prop_ok-p_Idx)
                break
            #if the proportion of protected group is achieved within the prefix, add rest as non-protected
            elif p_Idx == prop_ok:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx : dfpn_Idx + (prop_pn-pn_Idx) ])
                dfpn_Idx += (prop_pn-pn_Idx)
                break


        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [15]:
re_ranked_fair_for_RMSE = modified_FAIR(df_try, 4000, 0.0065, 0.40)
re_ranked_fair_for_RMSE.to_csv('re_ranked_fair_for_RMSE.csv')


### FAIR re-ranking of PLAIN XGBoost

In [18]:
RMSE_FAIR = pd.read_csv('RMSE_PLAIN.csv')
RMSE_FAIR = np.array_split(RMSE_FAIR, 200)

def fair_mange(df):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds','y_test'])
    for i in range(200):
        ress = modified_FAIR(df[i], 4000, 0.0065, 0.40)
        # ress.to_csv(f"re_ranked_{i}.csv")
        tom_res = tom_res.append(ress)
        # print(ress)
    
    tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    return tom_res.to_csv(f"RMSE_FAIR.csv")
fair_mange(RMSE_FAIR)



KeyboardInterrupt: 

Check ratio of women out of total

In [ ]:
# # sum(ress.Gender_bins)/len(ress)
#print(ress.Gender_bins.value_counts())
#print(len(ress[ress.Gender_bins == 1]))
#print(len(ress))
# print(len(ress[ress.Gender_bins == 1])/len(ress))
print('{0:.05f}'.format(len(o33[o33.Gender_bins == 1])/len(o33)))

0.39798


### FAIR re-ranking of Mergesort

In [66]:
df_merge_BIG_split[1]

,index,Idx,Gender_bins,Preds,COL_GRADE_AVG
4096,7598,4096,1,100.000000,98.666667
4097,4196,4097,1,97.666667,98.666667
4098,5527,4098,0,92.333333,100.000000
4099,4285,4099,1,91.000000,98.666667
4100,5852,4100,0,91.000000,89.333333
...,...,...,...,...,...
8187,6503,8187,1,36.333333,12.666667
8188,6184,8188,0,35.333333,7.333333
8189,4503,8189,0,33.666667,17.000000
8190,5217,8190,0,30.333333,24.000000


In [ ]:
# modified_FAIR(dfs[1], 500, 0.33, 0.40)

,index,Unnamed: 0,Idx,Gender_bins,Preds
0,0,0.0,8893.0,0.0,92.327812
1,1,2.0,5376.0,0.0,92.289169
2,2,3.0,10892.0,0.0,92.289169
3,3,1.0,7332.0,0.0,92.289169
4,4,11.0,11986.0,0.0,92.241760
...,...,...,...,...,...
490,293,494.0,4360.0,0.0,83.816170
491,294,499.0,10705.0,0.0,83.812927
492,295,498.0,11253.0,0.0,83.812927
493,296,497.0,9184.0,0.0,83.812927


In [ ]:
def fair_mange_merge(df):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds', 'COL_GRADE_AVG'])
    for i in range(200):
        ress = modified_FAIR(df[i], 500, 0.05, 0.40)
        # ress.to_csv(f"re_ranked_{i}.csv")
        tom_res = tom_res.append(ress)
        # print(ress)
    
    tom_res = tom_res.drop(['index'], axis=1)
    return tom_res.to_csv(f"200_Merge_fair.csv") 
fair_mange_merge(df_merge_BIG_split)


In [67]:
try_merge = pd.read_csv("200_Merge_fair.csv")

In [68]:
try_merge

,Unnamed: 0,not_4_use,Idx,Gender_bins,Preds,COL_GRADE_AVG
0,0,NaN,0.0,1.0,97.666667,98.666667
1,1,NaN,1.0,0.0,96.666667,97.333333
2,2,NaN,2.0,0.0,95.000000,97.000000
3,3,NaN,3.0,0.0,93.666667,96.333333
4,4,NaN,5.0,0.0,93.333333,100.000000
...,...,...,...,...,...,...
99995,495,NaN,815585.0,1.0,74.000000,86.333333
99996,496,NaN,815587.0,1.0,74.000000,85.333333
99997,497,NaN,815602.0,1.0,73.666667,91.000000
99998,498,NaN,815614.0,1.0,73.666667,75.000000


# Ny FA*IR - grouped fairness

In [5]:
# df_try = dfs[1]

df_try = df_merge_BIG_split[0]
df_try

,index,Idx,Gender_bins,Preds,COL_GRADE_AVG
0,1784,0,1,97.666667,98.666667
1,3721,1,0,96.666667,97.333333
2,1989,2,0,95.000000,97.000000
3,3830,3,0,93.666667,96.333333
4,2332,4,0,93.333333,98.000000
...,...,...,...,...,...
4091,3876,4091,1,37.333333,25.666667
4092,3966,4092,0,37.333333,45.000000
4093,1486,4093,0,36.666667,37.666667
4094,86,4094,1,36.333333,12.666667


In [28]:
#add STRATUM
# GUDF = pd.read_csv('GUDF.csv')
# GUDF["Idx"] = GUDF["Unnamed: 0"]
# GUDF

,Unnamed: 0,GENDER,STRATUM,SCHOOL_NAT,SCHOOL_TYPE,MAT_S11,CR_S11,CC_S11,BIO_S11,ENG_S11,QR_PRO,CR_PRO,CC_PRO,ENG_PRO,WC_PRO,COL_GRADE_AVG,HI_GRADE_AVG,GENDER_bin,Idx
0,0,F,Stratum_4,PRIVATE,ACADEMIC,71,81,61,86,82,71,93,71,93,79,85.666667,74.666667,1,0
1,1,F,Stratum_5,PRIVATE,ACADEMIC,83,75,66,100,88,97,38,86,98,78,74.000000,76.333333,1,1
2,2,M,Stratum_2,PRIVATE,ACADEMIC,52,49,38,46,42,17,1,18,43,22,20.666667,43.000000,0,2
3,3,F,Stratum_2,PRIVATE,ACADEMIC,56,55,51,64,73,65,35,76,80,48,63.666667,59.666667,1,3
4,4,M,Stratum_4,PRIVATE,ACADEMIC,80,65,76,85,92,94,94,98,100,71,97.333333,77.666667,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12406,12406,M,Stratum_2,PRIVATE,ACADEMIC,67,69,70,67,81,88,71,86,87,65,81.333333,73.333333,0,12406
12407,12407,M,Stratum_2,PUBLIC,ACADEMIC,58,57,61,63,53,46,39,44,11,0,31.333333,57.000000,0,12407
12408,12408,M,Stratum_2,PRIVATE,ACADEMIC,66,69,75,70,58,98,88,90,81,87,86.333333,67.333333,0,12408
12409,12409,F,Stratum_3,PRIVATE,ACADEMIC,53,69,64,59,52,60,80,51,8,42,46.333333,61.666667,1,12409


In [6]:
#Merge with Stratum column
GUDF = pd.read_csv('GUDF.csv')
GUDF["Idx"] = GUDF["Unnamed: 0"]
df_try = df_try.merge(GUDF[["Idx", "STRATUM"]], on="Idx")

#remove stratum = 0
df_try = df_try[df_try.STRATUM != "0"]

In [7]:
df_try

,index,Idx,Gender_bins,Preds,COL_GRADE_AVG,STRATUM
0,1784,0,1,97.666667,98.666667,Stratum_4
1,3721,1,0,96.666667,97.333333,Stratum_5
2,1989,2,0,95.000000,97.000000,Stratum_2
3,3830,3,0,93.666667,96.333333,Stratum_2
4,2332,4,0,93.333333,98.000000,Stratum_4
...,...,...,...,...,...,...
4091,3876,4091,1,37.333333,25.666667,Stratum_1
4092,3966,4092,0,37.333333,45.000000,Stratum_2
4093,1486,4093,0,36.666667,37.666667,Stratum_3
4094,86,4094,1,36.333333,12.666667,Stratum_3


In [267]:
##### Get ratio KUN stratum

# df_count = df_try[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count() #.drop(["0"], axis=0) # denne var oprindeligt. Den tager højde for både kvinder og stratum
df_count = df_try[["STRATUM","Preds"]].groupby(["STRATUM"], axis=0).count() #.drop(["0"], axis=0)   #denne tager altså kun højde for stratum

#len of df without STRATUM = 0
len_strats = df_count.Preds.sum()

#Divide df_counts with len
df_count["ratio"] = df_count.Preds/len_strats

df_count

,Preds,ratio
STRATUM,,
Stratum_1,615,0.150661
Stratum_2,1450,0.355218
Stratum_3,1374,0.336600
Stratum_4,442,0.108280
Stratum_5,136,0.033317
Stratum_6,65,0.015924


In [262]:
#### Antal candidates required in every prefix

k=200
antal_cand_der_kræves = []
for i in range(6):
    q=df_count.iloc[i][1]*k
    antal_cand_der_kræves.append(q)
antal_cand_der_kræves[0]

30.132288094071534

In [252]:
# df_try[(df_try.Gender_bins == 1) & (df_try.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
df_try.sort_values(by='Preds', ascending=False).reset_index()

,level_0,index,Idx,Gender_bins,Preds,COL_GRADE_AVG,STRATUM
0,0,1784,0,1,97.666667,98.666667,Stratum_4
1,1,3721,1,0,96.666667,97.333333,Stratum_5
2,2,1989,2,0,95.000000,97.000000,Stratum_2
3,3,3830,3,0,93.666667,96.333333,Stratum_2
4,4,2332,4,0,93.333333,98.000000,Stratum_4
...,...,...,...,...,...,...,...
4077,4091,3876,4091,1,37.333333,25.666667,Stratum_1
4078,4092,3966,4092,0,37.333333,45.000000,Stratum_2
4079,4093,1486,4093,0,36.666667,37.666667,Stratum_3
4080,4094,86,4094,1,36.333333,12.666667,Stratum_3


In [253]:
df_str_1 = df_try[(df_try.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
df_str_2 = df_try[(df_try.STRATUM =="Stratum_2")].sort_values(by='Preds', ascending=False).reset_index()
df_str_3 = df_try[(df_try.STRATUM =="Stratum_3")].sort_values(by='Preds', ascending=False).reset_index()
df_str_4 = df_try[(df_try.STRATUM =="Stratum_4")].sort_values(by='Preds', ascending=False).reset_index()
df_str_5 = df_try[(df_try.STRATUM =="Stratum_5")].sort_values(by='Preds', ascending=False).reset_index()
df_str_6 = df_try[(df_try.STRATUM =="Stratum_6")].sort_values(by='Preds', ascending=False).reset_index()

#Her are the means of the high school scores in each stratum. We see, that the difference in 1,2,3 
# are not large, and approximately twice as large between 4,5, and 6
mean_preds_str=[]
for i in range(1,7):
    b = eval('df_str_'+str(i)).Preds.mean()
    mean_preds_str.append(b)
mean_preds_str[0]




59.750135501355004

In [254]:
# here we are only including thoese that have preds higher than the distributions mean
df_str_1=df_str_1[(df_str_1.Preds > mean_preds_str[0])]
df_str_2=df_str_2[(df_str_2.Preds > mean_preds_str[1])]
df_str_3=df_str_3[(df_str_3.Preds > mean_preds_str[2])]
df_str_4=df_str_4[(df_str_4.Preds > mean_preds_str[3])]
df_str_5=df_str_5[(df_str_5.Preds > mean_preds_str[4])]
df_str_6=df_str_6[(df_str_6.Preds > mean_preds_str[5])]

længde=len(df_str_1)+len(df_str_2)+len(df_str_3)+len(df_str_4)+len(df_str_5)+len(df_str_6)

adjusted_ratios=[]
for i in range(1,7):
    r=len(eval('df_str_'+str(i)))/længde
    adjusted_ratios.append(r)
adjusted_ratios


[0.17552916881775943,
 0.37790397521941144,
 0.28910686628807436,
 0.10428497676819824,
 0.03768714506969541,
 0.015487867836861126]

In [270]:
0.17*200*0.2 #antal mennesker per 20% i en ranking af 200
k=200
size_prefix=0.2  #20% Dette er altså for 40 mennesker

antal_cand_der_kræves_i_prefix=[]

for i in range(6):
    w = df_count.ratio[i]*k*size_prefix
    antal_cand_der_kræves_i_prefix.append(round(w))
 
antal_cand_der_kræves_i_prefix

[6, 14, 13, 4, 1, 1]

In [259]:
for i in range(6):
    print(antal_cand_der_kræves[i]/200*100)

15.066144047035767
35.521803037726606
33.659970602645764
10.828025477707007
3.3317001469867713
1.5923566878980893


In [179]:
# For tester 

# antal_cand_der_kræves=[2, 2, 2, 2, 1, 1]

In [218]:
# # res=pd.DataFrame(columns=['Predss','STRATUM'])
# k=200
# n_prefix=int(0.2*k)

# Predss = []
# STRA = []

# tæller = 0


# current_amount_of_candidates = pd.DataFrame(columns=['1','2','3','4','5','6'], index=range(k))

# # for a in range(2):
# #     current_amount_of_candidates.loc[a]['1'] = 1
# #     current_amount_of_candidates.loc[a]['2'] = 1
# #     current_amount_of_candidates.loc[a]['3'] = 1
# # current_amount_of_candidates['4']=current_amount_of_candidates['4'].append(data)
# current_amount_of_candidates

# # df = df.append(data)

,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
195,NaN,NaN,NaN,NaN,NaN,NaN
196,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN


In [284]:

k=200
n_prefix=int(0.2*k)
Predss = []
STRA = []

tæller1 = 0
tæller2 = 0
tæller3 = 0
tæller4 = 0
tæller5 = 0
tæller6 = 0


In [295]:
for i in range(n_prefix):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [294]:

for i in range(n_prefix):
    for a, b, c, d, e, f in zip(df_str_1.Preds[:n_prefix],df_str_2.Preds[:n_prefix],df_str_3.Preds[:n_prefix],df_str_4.Preds[:n_prefix],df_str_5.Preds[:n_prefix], df_str_6.Preds[:n_prefix]):
        # print(a,b,c,d,e,f)
        m=max(a,b,c,d,e,f)
        # print(m)
        
        if m == a:
            # while antal_cand_der_kræves_i_prefix[0] > tæller1:
            Predss.append([a,'1'])
            tæller1 +=1
            break
            
     
           
        elif m == b:
            # while antal_cand_der_kræves_i_prefix[1] > tæller2:
            Predss.append([b,'2'])
            tæller2 +=1
            break

        elif m == c:
            # while antal_cand_der_kræves_i_prefix[2] > tæller3:
            Predss.append([c,'3'])
            tæller3 +=1
            break

        elif m == d:
            # while antal_cand_der_kræves_i_prefix[3] > tæller4:
            Predss.append([d,'4'])
            tæller4 +=1
            break

        elif m == e:
            # while antal_cand_der_kræves_i_prefix[4] > tæller5: 
            Predss.append([e,'5'])
            tæller5 +=1
            break

        elif m == f:
            # while antal_cand_der_kræves_i_prefix[5] > tæller6:
            Predss.append([f,'6'])
            tæller6 +=1
            break
        

# for j in range(n_prefix):
#     STRA.append(Predss[j][1])

Predss

TypeError: 'int' object is not iterable

In [234]:
antal_cand_der_kræves[0]

2

In [239]:

# #count=0
# for idx, item in enumerate(list):
#     print item
#     #count +=1
#     #if count % 10 == 0:
#     if (idx+1) % 10 == 0:
#         print 'did ten'


tæller1 = 0
tæller2 = 0
tæller3 = 0
tæller4 = 0
tæller5 = 0
tæller6 = 0

for i in range(10):
    for a, b, c, d, e, f in zip(df_str_1.Preds,df_str_2.Preds,df_str_3.Preds,df_str_4.Preds,df_str_5.Preds, df_str_6.Preds):
        # print(a,b,c,d,e,f)
        m=max(a,b,c,d,e,f)
      

        if m == a and antal_cand_der_kræves[0] > tæller1:#current_amount_of_candidates['1'].count():
            Predss.append([a,'1'])
            tæller1 +=1
            
           
        elif m == b and antal_cand_der_kræves[1] > tæller2:#current_amount_of_candidates['2'].count():
            Predss.append([b,'2'])
            tæller2 +=1

        elif m == c and antal_cand_der_kræves[2] < tæller3:#current_amount_of_candidates['3'].count():
            Predss.append([c,'3'])
            tæller3 +=1

        elif m == d and antal_cand_der_kræves[3] < tæller4:
            Predss.append([d,'4'])
            tæller4 +=1

        elif m == e and antal_cand_der_kræves[4] < tæller5:#current_amount_of_candidates['5'].count():
            Predss.append([e,'5'])
            tæller5 +=1
      

        elif m == f and antal_cand_der_kræves[5] < tæller6:#current_amount_of_candidates['6'].count():
            Predss.append([f,'6'])
            tæller6 +=1
        

# for j in range(n_prefix):
#     STRA.append(Predss[j][1])

Predss


[[97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [97.66666666666669, '4'],
 [95.0, '2'],
 [93.66666666666669, '2'],
 [89.33333333333333, '2'],
 [95.0, '2'],
 [93.66666666666669, '2'],
 [95.0, '2'],
 [93.66666666666669, '2'],
 [95.0, '2'],
 [93.66666666666669, '2'],
 [93.66666666666669, '2'],
 [89.33333333333333, '2']]

In [ ]:
# n_cand0 = np.array([0,0,0,5])
# cand_prop0 = np.array([1,2,3,4])

In [ ]:
# if n_cand0.any()<cand_prop0.any():
#     print("true")

In [ ]:
# q = [n_cand0[i]>cand_prop0[i] for i in range(len(cand_prop0))]

In [ ]:
# np.where([n_cand0[i]>cand_prop0[i] for i in range(len(cand_prop0))])[0]

array([3])

In [ ]:
# arr_list = ["df_s1_g0","df_s2_g0","df_s3_g0","df_s4_g0","df_s5_g0","df_s6_g0",
#             "df_s1_g1","df_s2_g1","df_s3_g1","df_s4_g1","df_s5_g1","df_s6_g1"]

In [ ]:
# eval(arr_list[int(max_arr_idx)]+"_idx")

In [ ]:
def KAT_grouped_FAIR(df, k, n_prefix):
    
    """" 
    Our grouped fair algorithm inspired by Zehlike's FA*IR for multiple grouped:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired proportion for each n prefix
    as percentage e.g. "0.1"=10%. 

    Output is result of a grouped fair top-k ranking with desired proportion for the whole df and in every prefix,
    and added "Stratum" column.
    
    """
    df = df_try.copy()
    #Merge with Stratum column
    GUDF = pd.read_csv('GUDF.csv')
    GUDF["Idx"] = GUDF["Unnamed: 0"]
    # df = df.merge(GUDF[["Idx", "STRATUM"]], on="Idx")
    #remove stratum = 0
    # df = df[df.STRATUM != "0"]


    #we subtract by 0, because we are using 0-index
    n_prefix = int(n_prefix*k)-1 #TODO måske prøv at fjerne -1
    # print(n_prefix)
    
    #new df to append new ranking to
    # print(df.columns)
    dfnew = pd.DataFrame(columns = df.columns)

In [60]:
def our_grouped_FAIR(df, k, n_prefix):
    
    """" 
    Our grouped fair algorithm inspired by Zehlike's FA*IR for multiple grouped:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired proportion for each n prefix
    as percentage e.g. "0.1"=10%. 

    Output is result of a grouped fair top-k ranking with desired proportion for the whole df and in every prefix,
    and added "Stratum-gender" column.
    
    """
    df = df_try.copy()
    #Merge with Stratum column
    GUDF = pd.read_csv('GUDF.csv')
    GUDF["Idx"] = GUDF["Unnamed: 0"]
    # df = df.merge(GUDF[["Idx", "STRATUM"]], on="Idx")
    #remove stratum = 0
    # df = df[df.STRATUM != "0"]


    #we subtract by 0, because we are using 0-index
    n_prefix = int(n_prefix*k)-1 #TODO måske prøv at fjerne -1
    # print(n_prefix)
    
    #new df to append new ranking to
    # print(df.columns)
    dfnew = pd.DataFrame(columns = df.columns)
    

    ## Split df in groups by gender-stratum: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #Gender = 0
    df_s1_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
    df_s2_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_2")].sort_values(by='Preds', ascending=False).reset_index()
    df_s3_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_3")].sort_values(by='Preds', ascending=False).reset_index()
    df_s4_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_4")].sort_values(by='Preds', ascending=False).reset_index()
    df_s5_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_5")].sort_values(by='Preds', ascending=False).reset_index()
    df_s6_g0 = df[(df.Gender_bins == 0) & (df.STRATUM =="Stratum_6")].sort_values(by='Preds', ascending=False).reset_index()
    
    #Gender = 1
    df_s1_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_1")].sort_values(by='Preds', ascending=False).reset_index()
    df_s2_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_2")].sort_values(by='Preds', ascending=False).reset_index()
    df_s3_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_3")].sort_values(by='Preds', ascending=False).reset_index()
    df_s4_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_4")].sort_values(by='Preds', ascending=False).reset_index()
    df_s5_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_5")].sort_values(by='Preds', ascending=False).reset_index()
    df_s6_g1 = df[(df.Gender_bins == 1) & (df.STRATUM =="Stratum_6")].sort_values(by='Preds', ascending=False).reset_index()
    
    
    ## Get ratio of each stratum-gender pair in the initial dataframe
    
    #count number of candidates in initial df
    df_count = df[["STRATUM","Gender_bins","Preds"]].groupby(["STRATUM", "Gender_bins"], axis=0).count()


    #Divide df_counts with len
    df_count["ratio"] = df_count.Preds/len(df) #then df_count[0][0] will be (Stratum_1, 0) and it's proportion
            
    #desired number of protected candidates in each n batch
    prop_s1_g0 = int(n_prefix*df_count["ratio"].iloc[0])
    prop_s2_g0 = int(n_prefix*df_count["ratio"].iloc[2])
    prop_s3_g0 = int(n_prefix*df_count["ratio"].iloc[4])
    prop_s4_g0 = int(n_prefix*df_count["ratio"].iloc[6])
    prop_s5_g0 = int(n_prefix*df_count["ratio"].iloc[8])
    prop_s6_g0 = int(n_prefix*df_count["ratio"].iloc[10])

    prop_s1_g1 = int(n_prefix*df_count["ratio"].iloc[1])
    prop_s2_g1 = int(n_prefix*df_count["ratio"].iloc[3])
    prop_s3_g1 = int(n_prefix*df_count["ratio"].iloc[5])
    prop_s4_g1 = int(n_prefix*df_count["ratio"].iloc[7])
    prop_s5_g1 = int(n_prefix*df_count["ratio"].iloc[9])
    prop_s6_g1 = int(n_prefix*df_count["ratio"].iloc[11])

    
    # array with proportions 
    cand_prop = np.array([prop_s1_g0, prop_s2_g0, prop_s3_g0, prop_s4_g0, prop_s5_g0, prop_s6_g0,
                         prop_s1_g1, prop_s2_g1, prop_s3_g1, prop_s4_g1, prop_s5_g1, prop_s6_g1])
    
    print(cand_prop)
    
    ## For-loop for creating ranking in batches of prefix_n through k
    
    #index in each dataframe    
    df_s1_g0_idx = 0
    df_s2_g0_idx = 0
    df_s3_g0_idx = 0
    df_s4_g0_idx = 0
    df_s5_g0_idx = 0
    df_s6_g0_idx = 0  
    
    df_s1_g1_idx = 0
    df_s2_g1_idx = 0
    df_s3_g1_idx = 0
    df_s4_g1_idx = 0
    df_s5_g1_idx = 0
    df_s6_g1_idx = 0 

    arr_list = ["df_s1_g0","df_s2_g0","df_s3_g0","df_s4_g0","df_s5_g0","df_s6_g0",
                "df_s1_g1","df_s2_g1","df_s3_g1","df_s4_g1","df_s5_g1","df_s6_g1"]

    
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        #number of candidates in prefix of each group
        s1_g0_idx = 0
        s2_g0_idx = 0
        s3_g0_idx = 0
        s4_g0_idx = 0
        s5_g0_idx = 0
        s6_g0_idx = 0  
    
        s1_g1_idx = 0
        s2_g1_idx = 0
        s3_g1_idx = 0
        s4_g1_idx = 0
        s5_g1_idx = 0
        s6_g1_idx = 0 
        #print("i=",i)
        
        # print(s4_g0_idx)
        
        n_cand = np.array([s1_g0_idx,s2_g0_idx,s3_g0_idx,s4_g0_idx,s5_g0_idx,s6_g0_idx,
                          s1_g1_idx,s2_g1_idx,s3_g1_idx,s4_g1_idx,s5_g1_idx,s6_g1_idx])
        
        for j in range(n_prefix):
            
        #Check if number of candidates is less than proportion
            
            if all([n_cand[ki]<=cand_prop[ki] for ki in range(len(cand_prop))]):
                # print("if true")
                arr_idx = np.array([df_s1_g0.Preds.iloc[df_s1_g0_idx],df_s2_g0.Preds.iloc[df_s2_g0_idx],
                                    df_s3_g0.Preds.iloc[df_s3_g0_idx],df_s4_g0.Preds.iloc[df_s4_g0_idx],
                                    df_s5_g0.Preds.iloc[df_s5_g0_idx],df_s6_g0.Preds.iloc[df_s6_g0_idx],
                                    df_s1_g1.Preds.iloc[df_s1_g1_idx],df_s2_g1.Preds.iloc[df_s2_g1_idx],
                                    df_s3_g1.Preds.iloc[df_s3_g1_idx],df_s4_g1.Preds.iloc[df_s4_g1_idx],
                                    df_s5_g1.Preds.iloc[df_s5_g1_idx],df_s6_g1.Preds.iloc[df_s6_g1_idx]])
                max_arr_idx = np.where(arr_idx == np.amax(arr_idx))[0][0] #make random choice da flere har den samme pred val. # TODO søg på amax
                # print("arr_idx",arr_idx)
                
                dfnew = dfnew.append(eval(arr_list[max_arr_idx]).loc[eval(arr_list[max_arr_idx]+"_idx")])
                print(dfnew) # DER SKER NOGET MYSTISK HER

                exec(arr_list[int(max_arr_idx)]+"_idx" '+=' '1')
                print("exec", eval(arr_list[int(max_arr_idx)]+"_idx"))
                # 
                
                exec(arr_list[int(max_arr_idx)][3:]+"_idx" '+=' '1')
                print(s4_g0_idx)

                #break

            #else if num of candidates in n_prefix is equal to or higher than cand_proportion,
            #only add max of the others.

            else:
                print("else true")
                list_greater = np.where([n_cand[ki]>=cand_prop[ki] for ki in range(len(cand_prop))])[0]
                arr_idx = np.array([df_s1_g0.Preds.iloc[df_s1_g0_idx],df_s2_g0.Preds.iloc[df_s2_g0_idx],
                                    df_s3_g0.Preds.iloc[df_s3_g0_idx],df_s4_g0.Preds.iloc[df_s4_g0_idx],
                                    df_s5_g0.Preds.iloc[df_s5_g0_idx],df_s6_g0.Preds.iloc[df_s6_g0_idx],
                                    df_s1_g1.Preds.iloc[df_s1_g1_idx],df_s2_g1.Preds.iloc[df_s2_g1_idx],
                                    df_s3_g1.Preds.iloc[df_s3_g1_idx],df_s4_g1.Preds.iloc[df_s4_g1_idx],
                                    df_s5_g1.Preds.iloc[df_s5_g1_idx],df_s6_g1.Preds.iloc[df_s6_g1_idx]])
                arr_idx = np.delete(arr_idx, list_greater)
                
                max_arr_idx = np.where(arr_idx == np.amax(arr_idx))[0][0]
                dfnew = dfnew.append(eval(arr_list[max_arr_idx]).loc[eval(arr_list[max_arr_idx]+"_idx")])

                exec(arr_list[max_arr_idx]+"_idx" '+=' '1')
                exec(arr_list[max_arr_idx][3:]+"_idx" '+=' '1')
                #break



        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [58]:
df_try

,index,Idx,Gender_bins,Preds,COL_GRADE_AVG,STRATUM
0,1784,0,1,97.666667,98.666667,Stratum_4
1,3721,1,0,96.666667,97.333333,Stratum_5
2,1989,2,0,95.000000,97.000000,Stratum_2
3,3830,3,0,93.666667,96.333333,Stratum_2
4,2332,4,0,93.333333,98.000000,Stratum_4
...,...,...,...,...,...,...
4091,3876,4091,1,37.333333,25.666667,Stratum_1
4092,3966,4092,0,37.333333,45.000000,Stratum_2
4093,1486,4093,0,36.666667,37.666667,Stratum_3
4094,86,4094,1,36.333333,12.666667,Stratum_3


In [61]:
df_grouped_ranked = our_grouped_FAIR(df_try, 1000, 0.2)

[17 41 37 12  4  1 12 28 29  9  2  1]
0
if true
arr_idx [81.66666667 95.         88.         93.33333333 96.66666667 93.33333333
 88.33333333 89.33333333 89.33333333 97.66666667 91.         89.        ]
  index Idx Gender_bins      Preds  COL_GRADE_AVG    STRATUM  level_0
0  1784   0           1  97.666667      98.666667  Stratum_4      0.0
exec 0
0
if true
arr_idx [81.66666667 95.         88.         93.33333333 96.66666667 93.33333333
 88.33333333 89.33333333 89.33333333 97.66666667 91.         89.        ]
  index Idx Gender_bins      Preds  COL_GRADE_AVG    STRATUM  level_0
0  1784   0           1  97.666667      98.666667  Stratum_4      0.0
0  1784   0           1  97.666667      98.666667  Stratum_4      0.0
exec 0
0
if true
arr_idx [81.66666667 95.         88.         93.33333333 96.66666667 93.33333333
 88.33333333 89.33333333 89.33333333 97.66666667 91.         89.        ]
  index Idx Gender_bins      Preds  COL_GRADE_AVG    STRATUM  level_0
0  1784   0           1  97.66666

In [ ]:
df_grouped_ranked

,index,Unnamed: 0,Idx,Gender_bins,Preds,STRATUM
0,0,0,8893,0,92.327812,Stratum_4
1,1,0,8893,0,92.327812,Stratum_4
2,2,0,8893,0,92.327812,Stratum_4
3,3,0,8893,0,92.327812,Stratum_4
4,4,0,8893,0,92.327812,Stratum_4
...,...,...,...,...,...,...
485,0,0,8893,0,92.327812,Stratum_4
486,0,0,8893,0,92.327812,Stratum_4
487,0,0,8893,0,92.327812,Stratum_4
488,0,0,8893,0,92.327812,Stratum_4


## Save this as a reranking and plot the entropy and utility

In [ ]:
# re_ranked_WO = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_WO.to_csv("re_ranked_WO.csv")

# re_ranked_PLAIN = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_PLAIN.to_csv("re_ranked_PLAIN.csv")